In [87]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [88]:
# Load 
sales_data = pd.read_csv('../data/sales_data.csv')


# clean
sales_data.dropna(inplace=True)

sales_data.columns

Index(['StoreID', 'ProductID', 'Date', 'Sales', 'Customers', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'day_of_week', 'hour_of_day',
       'Demand'],
      dtype='object')

In [89]:
# Features engineering 
sales_data['day_of_week'] = pd.to_datetime(sales_data['Date']).dt.dayofweek
sales_data['hour_of_day'] = pd.to_datetime(sales_data['Date']).dt.hour


In [90]:
sales_data

,StoreID,ProductID,Date,Sales,Customers,Promo,StateHoliday,SchoolHoliday,day_of_week,hour_of_day,Demand
0,52,959,2024-11-12,71.02,96,0,0,0,1,0,55.10200
1,93,717,2024-09-03,673.91,39,0,0,0,1,0,86.89100
2,15,641,2024-01-05,786.28,10,0,0,0,4,0,83.62800
3,72,361,2024-07-10,496.63,97,0,0,0,2,0,98.16300
4,61,539,2024-07-16,787.65,18,1,0,0,1,0,105.31800
...,...,...,...,...,...,...,...,...,...,...,...
199995,42,434,2024-10-18,205.22,34,1,0,0,4,0,45.02640
199996,11,74,2024-09-27,497.40,41,1,0,0,4,0,84.28800
199997,71,414,2024-09-17,898.46,97,1,0,1,1,0,149.41368
199998,54,570,2024-08-16,440.24,31,0,0,0,4,0,59.52400


In [91]:
# calculate the damand column 
def cal_demand(row):
    base_demand = row['Sales'] * 0.1 + row['Customers'] * 0.5
    promo_factor = 1.2 if row['Promo'] == 1 else 1.0
    StateHoliday = 0.8 if row['StateHoliday'] == 1 else 1.0
    SchoolHoliday = 0.9 if row['SchoolHoliday'] == 1 else 1.0
    demand = base_demand * promo_factor * StateHoliday * SchoolHoliday
    return demand 



In [92]:
# Add to remaing data
sales_data['Demand'] = sales_data.apply(cal_demand, axis=1)



In [93]:
sales_data.to_csv("C:/Users/PAUL/Desktop/PriceFlex/data/sales_data.csv", index=False)
print("Demand Column added")

Demand Column added


In [94]:
sales_data

,StoreID,ProductID,Date,Sales,Customers,Promo,StateHoliday,SchoolHoliday,day_of_week,hour_of_day,Demand
0,52,959,2024-11-12,71.02,96,0,0,0,1,0,55.10200
1,93,717,2024-09-03,673.91,39,0,0,0,1,0,86.89100
2,15,641,2024-01-05,786.28,10,0,0,0,4,0,83.62800
3,72,361,2024-07-10,496.63,97,0,0,0,2,0,98.16300
4,61,539,2024-07-16,787.65,18,1,0,0,1,0,105.31800
...,...,...,...,...,...,...,...,...,...,...,...
199995,42,434,2024-10-18,205.22,34,1,0,0,4,0,45.02640
199996,11,74,2024-09-27,497.40,41,1,0,0,4,0,84.28800
199997,71,414,2024-09-17,898.46,97,1,0,1,1,0,149.41368
199998,54,570,2024-08-16,440.24,31,0,0,0,4,0,59.52400


In [95]:
competitor_data = pd.read_csv("C:/Users/PAUL/Desktop/PriceFlex/data/competitor_data.csv")

sales_data['Date'] = pd.to_datetime(sales_data['Date'])
competitor_data['Date'] = pd.to_datetime(competitor_data['Date'])

# Create a dictionary of fast lookup
competitor_lookup = {}
for _, row in competitor_data.iterrows():
    key = (row['ProductID'], row['Date'])
    competitor_lookup[key] = row['CompetitorPrice']

# Function for get competitor pricing 
def get_competitor_price(product_id, date):
    key = (product_id, pd.to_datetime(date))
    return competitor_lookup.get(key, None)

# Add columns
sales_data['CompetitorPrice'] = sales_data.apply(
    lambda row: get_competitor_price(row['ProductID'], row['Date']), axis=1
)

# Filling miissing  values
mean_comptitor_price = sales_data['CompetitorPrice'].mean()
sales_data.loc[sales_data['CompetitorPrice'].isna(),'CompetitorPrice'] = mean_comptitor_price

sales_data.head(14)


,StoreID,ProductID,Date,Sales,Customers,Promo,StateHoliday,SchoolHoliday,day_of_week,hour_of_day,Demand,CompetitorPrice
0,52,959,2024-11-12,71.02,96,0,0,0,1,0,55.1020,251.938357
1,93,717,2024-09-03,673.91,39,0,0,0,1,0,86.8910,251.938357
2,15,641,2024-01-05,786.28,10,0,0,0,4,0,83.6280,251.938357
3,72,361,2024-07-10,496.63,97,0,0,0,2,0,98.1630,251.938357
4,61,539,2024-07-16,787.65,18,1,0,0,1,0,105.3180,251.938357
5,21,61,2024-03-10,976.50,50,0,0,0,6,0,122.6500,251.938357
6,83,123,2024-07-11,901.55,43,0,0,0,3,0,111.6550,251.938357
7,87,963,2024-02-02,806.43,18,0,0,0,4,0,89.6430,251.938357
8,75,640,2024-02-24,308.01,98,1,0,0,5,0,95.7612,251.938357
9,75,869,2024-07-08,671.39,13,0,0,1,0,0,66.2751,408.990000


In [100]:
# Normalize and encode
numerical_features = ['Sales', 'CompetitorPrice']
categorical_features = ['day_of_week', 'hour_of_day']

In [101]:
numerical_transformer = Pipeline(steps=[
    ('scaler', MinMaxScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [102]:
X = sales_data[['Sales', 'CompetitorPrice', 'day_of_week', 'hour_of_day']]
y = sales_data['Demand']



In [ ]:
X_preprocessed = preprocessor.fit_transform(X)

# save preprocessed data
np.save('../data/processed/X_preprocessed.npy', X_preprocessed)
np.save('../data/preprocessed/y.npy', y)

# Save preprocessor
import joblib

X = joblib.dump('../data/processed/preprocessor.pkl', preprocessor)